# Molecular Mechanics Force Field

There is several ways to build a molecule object within VeloxChem using for instance a smile string, directly from an xyz or pdb file or by providing and xyz string. This is explicited below with few examples. The 3D structure of the molecular object can be visualized with the show function, and the atom indices can be shown optionally. 

In [1]:
import veloxchem as vlx

In [2]:
molecule = vlx.Molecule.read_smiles("NC(C(=O)O)Cc1ccccc1")
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
molecule = vlx.Molecule.read_xyz_file("../data/phenylalanine.xyz")
molecule.show(atom_indices=True)
protein = vlx.Molecule.read_pdb_file("../data/1ajj.pdb")
protein.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.